In [146]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model

trained_model_dir = "F:/FNC 프로젝트/V-SMR/model_24DEC24_lossfunc_comparison_250715"
trained_model_dir_server = "/home/jskwon/Desktop/projects/ongoing/NTC_SUB3/assy3d_BOC/model_24DEC24_lossfunc_comparison_250715"

데이터셋
- 250314 버전... 이 아니네 더 옛날꺼네
- 입력1: XS, shape:(24,5,5,7)
- 입력2: Rod, shape: (24,5,5)
- 출력: P3D_assy, shape:(24,5,5)

In [147]:
os.chdir(trained_model_dir_server)

# x_lp_3d = np.load("train_dataset_model_loss_function_evaluation_250730/filtered_LP_FA3D_09MAY24.npy", allow_pickle=True)
# x_rod_3d = np.load("train_dataset_model_loss_function_evaluation_250730/assywise_rod_map_layers_09MAY24.npy", allow_pickle=True) 
# y_p3d = np.load("train_dataset_model_loss_function_evaluation_250730/filtered_FA_p3d_09MAY24.npy", allow_pickle=True)

x_lp_3d = np.load("../train_dataset/filtered_LP_FA3D_09MAY24.npy", allow_pickle=True)
x_rod_3d = np.load("../train_dataset/assywise_rod_map_layers_09MAY24.npy", allow_pickle=True) 
y_p3d = np.load("../train_dataset/filtered_FA_p3d_09MAY24.npy", allow_pickle=True)

x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(x_lp_3d,x_rod_3d, y_p3d, test_size=0.1, random_state=42)
x1_train, x1_valid, x2_train, x2_valid, y_train, y_valid = train_test_split(x1_train,x2_train, y_train, test_size=0.05, random_state=22)

# 데이터 타입 변환
x1_train = np.array(x1_train, dtype=np.float32)
x2_train = np.array(x2_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
x1_valid = np.array(x1_valid, dtype=np.float32)
x2_valid = np.array(x2_valid, dtype=np.float32)
y_valid = np.array(y_valid, dtype=np.float32)
x1_test = np.array(x1_test, dtype=np.float32)
x2_test = np.array(x2_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

모델

In [58]:
model_path = "model_rod3d_huber.h5"

model_temp = load_model(model_path, compile=False)

In [59]:
pred = model_temp.predict([x1_test,x2_test])
pred_squeezed = np.squeeze(pred, axis=-1)

118/118 [==============================] - 2s 15ms/step


In [28]:
pred_squeezed.shape


(3754, 24, 5, 5)

### 0. metrices

In [188]:
model_path = "model_rod3d_try5th_log_cosh.h5"

#"model_rod3d_try5th_smape.h5"
model_temp = load_model(model_path, compile=False)
pred = model_temp.predict([x1_test,x2_test])
pred_squeezed = np.squeeze(pred, axis=-1)
pred_symmetric = apply_symmetry(pred_squeezed)

calculate_global_voxel_errors(y_test, pred_symmetric)
calculate_peak_value_errors(y_test, pred_symmetric)
calculate_peak_accuracy_symmetric(y_test, pred_symmetric)
calculate_peak_accuracy_z_tolerant(y_test, pred_symmetric)
calculate_peak_accuracy_xy_plane(y_test, pred_symmetric)

118/118 [==============================] - 2s 12ms/step
관심 영역(ROI) 내 전역 평균 절대 백분율 오차 (Global MAPE): 1.230%
관심 영역(ROI) 내 전역 최대 절대 백분율 오차 (Global MaxAPE): 29.097%
관심 영역(ROI) 내 Peak 예측의 전역 평균 절대 백분율 오차 (Global MAPE): 0.828%
관심 영역(ROI) 내 Peak 예측의 전역 최대 절대 백분율 오차 (Global MaxAPE): 7.334%
Peak 위치 (z,y,x) 예측 정확도: 89.40%
Peak 위치 (z+-1,y,x) 예측 정확도: 96.30%
Peak 위치 (:,y,x) 예측 정확도: 96.30%


96.29728289824187

118/118 [==============================] - 2s 12ms/step
관심 영역(ROI) 내 전역 평균 절대 백분율 오차 (Global MAPE): 1.230%
관심 영역(ROI) 내 전역 최대 절대 백분율 오차 (Global MaxAPE): 29.097%
관심 영역(ROI) 내 Peak 예측의 전역 평균 절대 백분율 오차 (Global MAPE): 0.828%
관심 영역(ROI) 내 Peak 예측의 전역 최대 절대 백분율 오차 (Global MaxAPE): 7.334%
89.39797549280767

### 1.전역 평균&최대 상대 퍼센트 오차

In [5]:
def calculate_global_voxel_errors(y_true, y_pred, voxel_type = "assy", epsilon=1e-08):
    # voxel_type = "assy" / "pin"
    if voxel_type == "assy":
        # --- 마스크 생성 ---
        mask_radial = np.ones((5, 5), dtype=bool)
        mask_radial[3, 4] = False
        mask_radial[4, 3] = False
        mask_radial[4, 4] = False
        # 처음부터 boolean으로 만들면 .astype(bool)이 필요 없습니다.

        # (24, 5, 5) 형태로 복셀 마스크 생성
        mask_voxel = np.tile(mask_radial, (24, 1, 1))

        # --- MAPE 계산 ---

        # 1. 마스크를 사용해 관심 영역의 데이터만 추출
        y_test_masked = y_test[:, mask_voxel] # y_test/pred_squeezed의 각 샘플에 동일한 마스크를 적용
        pred_masked = pred_squeezed[:, mask_voxel]

        # 2. 0으로 나누기 방지를 위한 epsilon 설정
        epsilon = epsilon

        # 3. 마스킹된 데이터에 대해 APE(절대 백분율 오차) 계산
        percent_errors = np.abs((y_test_masked - pred_masked) / (y_test_masked + epsilon)) * 100

        # 4. 모든 APE의 평균을 내어 최종 MAPE(평균 절대 백분율 오차) 계산
        global_mape = np.mean(percent_errors)
        global_maxape = np.max(percent_errors)

        print(f"관심 영역(ROI) 내 전역 평균 절대 백분율 오차 (Global MAPE): {global_mape:.3f}%")
        print(f"관심 영역(ROI) 내 전역 최대 절대 백분율 오차 (Global MaxAPE): {global_maxape:.3f}%")
        
        return global_mape, global_maxape
    
        


In [111]:
calculate_global_voxel_errors(y_test, pred_squeezed)

관심 영역(ROI) 내 전역 평균 절대 백분율 오차 (Global MAPE): 1.43%
관심 영역(ROI) 내 전역 최대 절대 백분율 오차 (Global MaxAPE): 33.83%


### 2. peak value error
- np.where()을 사용하면, 해당 value의 위치 인덱스는 반환됨

NumPy의 axis를 사용하는 벡터화(Vectorized) 방법 (for문 루프 대체)

In [6]:
def calculate_peak_value_errors(y_true, y_pred, voxel_type = "assy", epsilon=1e-08):

    # axis=(1, 2, 3)을 지정하여 해당 축들을 기준으로 최댓값을 계산
    # (3754, 24, 5, 5) -> (3754,)의 배열이 생성
    peak_values_pred_vectorized = np.max(pred_squeezed, axis=(1, 2, 3))
    peak_values_true_vectorized = np.max(y_test, axis=(1, 2, 3))

    # print(f"예측값 Peak 배열 Shape: {peak_values_pred_vectorized.shape}")
    # print(f"실제값 Peak 배열 Shape: {peak_values_true_vectorized.shape}")

    percent_errors_peak_values = abs((peak_values_true_vectorized-peak_values_pred_vectorized)/peak_values_true_vectorized+epsilon)*100
    peak_value_mape = np.mean(percent_errors_peak_values)
    peak_value_maxape = np.max(percent_errors_peak_values)

    print(f"관심 영역(ROI) 내 Peak 예측의 전역 평균 절대 백분율 오차 (Global MAPE): {peak_value_mape:.3f}%")
    print(f"관심 영역(ROI) 내 Peak 예측의 전역 최대 절대 백분율 오차 (Global MaxAPE): {peak_value_maxape:.3f}%")

    return peak_value_mape, peak_value_maxape

In [113]:
calculate_peak_value_errors(y_test, pred_squeezed)

관심 영역(ROI) 내 Peak 예측의 전역 평균 절대 백분율 오차 (Global MAPE): 0.78%
관심 영역(ROI) 내 Peak 예측의 전역 최대 절대 백분율 오차 (Global MaxAPE): 8.12%


In [56]:
peak_value_mape, peak_value_maxape

(0.78475463, 8.11692)

In [63]:
pred_reshaped.shape

(3754, 600)

In [67]:
pred_squeezed.shape[1:]

(24, 5, 5)

In [69]:
flat_indices_pred

array([475, 412, 412, ..., 450, 428, 428], dtype=int64)

In [71]:
voxel_indices_pred

(array([19, 16, 16, ..., 18, 17, 17], dtype=int64),
 array([0, 2, 2, ..., 0, 0, 0], dtype=int64),
 array([0, 2, 2, ..., 0, 3, 3], dtype=int64))

In [89]:
# 1~3 과정: 1차원 배열로 펼친 후 최대값의 인덱스를 찾은 후 다시 3차원 인덱스로 재변환
    # np.prod(): 리스트의 원소곱을 반환하는 함수이나, 단순히 차원을 곱하기 위해 사용, 24*5*5
    # np.argmax(): 해당 축방향을 따라서 최대값의 인덱스를 반환
        # 3차원 축방향에 대해서 3번 쓰는것보다, flatten한뒤 복구하는게 편하겠지

# 1. (24, 5, 5) 복셀을 1차원으로 펼쳐서 (3754, 600) 형태로 만듭니다.
#    (24 * 5 * 5 = 600)
num_samples = pred_squeezed.shape[0]
voxel_size = np.prod(pred_squeezed.shape[1:]) # 24*5*5 = 600
pred_reshaped = pred_squeezed.reshape(num_samples, voxel_size) # (3754, 600)
y_test_reshaped =y_test.reshape(num_samples, voxel_size)

# 2. 펼쳐진 축(axis=1)을 따라 각 샘플의 최대값 인덱스를 반환 (Returns the indices of the maximum values along an axis.)
    # 결과는 각 샘플별 peak의 1차원 인덱스 배열 (shape: (3754,))
    # 예시: [475, 412, 412, ..., 450, 428, 428] -> 1차원으로 변환했을때의 최대값 인덱스
flat_indices_pred = np.argmax(pred_reshaped, axis=1) 
flat_indices_true = np.argmax(y_test_reshaped, axis=1)

# 3. 1차원 인덱스를 다시 (z, y, x) 형태의 3차원 복셀 인덱스로 변환
#    np.unravel_index는 인덱스 배열 전체에 대해 한 번에 연산 가능
voxel_indices_pred = np.unravel_index(flat_indices_pred, pred_squeezed.shape[1:]) # pred_squeezed.shape[1:] = (24, 5, 5)
voxel_indices_true = np.unravel_index(flat_indices_true, y_test.shape[1:])

# voxel_indices_pred는 (z_indices, y_indices, x_indices) 형태의 튜플입니다.
# 각 요소는 (3754,) 크기의 배열

# 4. (샘플수, 3)의 형태이며, 3은 (z,y,x)
peak_indices_array_pred = np.stack(voxel_indices_pred, axis=1)
peak_indices_array_true = np.stack(voxel_indices_true, axis=1)


# --- 인덱스 좌표(z,y,x)가 동일한지 비교 ---

# 1. 요소별 비교: (3754, 3) 모양의 boolean 배열 생성
#    [[True, True, True],   <- 0번 샘플: z,y,x 모두 일치
#     [True, True, False],  <- 1번 샘플: x가 불일치
comparison_array = (peak_indices_array_pred == peak_indices_array_true)
# 2. 각 행(axis=1)의 모든 요소가 True인지 확인
#    결과는 (3754,) shape의 최종 boolean 배열
matches = np.all(comparison_array, axis=1)

accuracy_peak_location = np.mean(matches) * 100



In [90]:
accuracy_peak_location

74.88012786361215

### 3. peak location accuracy

함수 2개 추가됨, 대칭 고려버전은 그대로 사용하면됨
- z축을 기준으로 특정 지점의 한칸 아래/위에 있는 경우 (Axial Tolerance)
- 2차원 좌표(x,y)가 일치하는 경우 (XY plane Projection)

In [149]:
def calculate_peak_accuracy_z_tolerant(y_true, y_pred):
    """
    (신규 지표 1) Z축 근접성을 허용하여 Peak 위치 예측 정확도를 계산합니다.
    기존 대칭 정확도 조건에 더해, (x,y)가 일치하고 z가 ±1 차이 나도 정답으로 인정합니다.

    Args:
        y_true (np.ndarray): 실제값 배열.
        y_pred (np.ndarray): 예측값 배열.

    Returns:
        float: Z축 근접성을 허용한 Peak 위치 예측 정확도 (%).
    """
    # --- 1. Reshape & Find 1D/3D indices (공통 로직) ---
    num_samples = y_pred.shape[0]
    voxel_shape = y_pred.shape[1:]
    
    pred_reshaped = y_pred.reshape(num_samples, -1)
    y_true_reshaped = y_true.reshape(num_samples, -1)

    flat_indices_pred = np.argmax(pred_reshaped, axis=1)
    flat_indices_true = np.argmax(y_true_reshaped, axis=1)

    peak_indices_pred = np.stack(np.unravel_index(flat_indices_pred, voxel_shape), axis=1)
    peak_indices_true = np.stack(np.unravel_index(flat_indices_true, voxel_shape), axis=1)

    # --- 2. 기존 대칭 정확도 계산 ---
    pred_z, pred_y, pred_x = peak_indices_pred[:, 0], peak_indices_pred[:, 1], peak_indices_pred[:, 2]
    true_z, true_y, true_x = peak_indices_true[:, 0], peak_indices_true[:, 1], peak_indices_true[:, 2]

    peak_indices_true_symmetric = np.stack([true_z, true_x, true_y], axis=1)
    matches_direct = np.all(peak_indices_pred == peak_indices_true, axis=1)
    matches_symmetric = np.all(peak_indices_pred == peak_indices_true_symmetric, axis=1)
    base_matches = np.logical_or(matches_direct, matches_symmetric)

    # --- 3. Z축 근접 허용 조건 추가 ---
    # 조건 A: (y,x) 좌표가 직접 또는 대칭으로 일치하는가?
    xy_match_direct = (pred_y == true_y) & (pred_x == true_x)
    xy_match_symmetric = (pred_y == true_x) & (pred_x == true_y)
    xy_match_any = np.logical_or(xy_match_direct, xy_match_symmetric)
    
    # 조건 B: z 좌표가 위 또는 아래로 인접하는가?
    z_is_adjacent = np.logical_or(pred_z == true_z - 1, pred_z == true_z + 1)

    # Z축 근접 정답: 조건 A와 조건 B를 모두 만족
    matches_z_tolerant = np.logical_and(xy_match_any, z_is_adjacent)

    # --- 4. 최종 정확도 계산 ---
    # 기본 정답 또는 Z축 근접 정답 중 하나라도 만족하면 최종 정답
    final_matches = np.logical_or(base_matches, matches_z_tolerant)
    
    accuracy = np.mean(final_matches) * 100
    
    print(f"Peak 위치 (z+-1,y,x) 예측 정확도: {accuracy:.2f}%")
    
    return accuracy

In [150]:
def calculate_peak_accuracy_xy_plane(y_true, y_pred):
    """
    (신규 지표 2) Z축을 무시하고 XY 평면의 대칭성만으로 Peak 위치 정확도를 계산합니다.
    예측 피크의 (y,x) 좌표가 실제 피크의 (y,x) 또는 (x,y)와 일치하면 정답으로 처리합니다.

    Args:
        y_true (np.ndarray): 실제값 배열.
        y_pred (np.ndarray): 예측값 배열.

    Returns:
        float: XY 평면 투영 Peak 위치 예측 정확도 (%).
    """
    # --- 1. Reshape & Find 1D/3D indices (공통 로직) ---
    num_samples = y_pred.shape[0]
    voxel_shape = y_pred.shape[1:]
    
    pred_reshaped = y_pred.reshape(num_samples, -1)
    y_true_reshaped = y_true.reshape(num_samples, -1)

    flat_indices_pred = np.argmax(pred_reshaped, axis=1)
    flat_indices_true = np.argmax(y_true_reshaped, axis=1)

    peak_indices_pred = np.stack(np.unravel_index(flat_indices_pred, voxel_shape), axis=1)
    peak_indices_true = np.stack(np.unravel_index(flat_indices_true, voxel_shape), axis=1)

    # --- 2. XY 평면 좌표만 추출 ---
    pred_y, pred_x = peak_indices_pred[:, 1], peak_indices_pred[:, 2]
    true_y, true_x = peak_indices_true[:, 1], peak_indices_true[:, 2]

    # --- 3. XY 평면에서의 대칭 정확도 계산 ---
    # 경우 1: (y,x) 좌표가 직접 일치
    matches_direct_xy = (pred_y == true_y) & (pred_x == true_x)
    
    # 경우 2: (y,x) 좌표가 대각 대칭으로 일치 (pred(y,x) == true(x,y))
    matches_symmetric_xy = (pred_y == true_x) & (pred_x == true_y)

    # 두 경우 중 하나라도 만족하면 최종 정답
    final_matches = np.logical_or(matches_direct_xy, matches_symmetric_xy)
    
    accuracy = np.mean(final_matches) * 100
    
    print(f"Peak 위치 (:,y,x) 예측 정확도: {accuracy:.2f}%")
    
    return accuracy

대칭 고려버전
- 둘다 True로 판단 (z, y, x) == (z, x, y)

In [151]:
def calculate_peak_accuracy_symmetric(y_true, y_pred):
    """
    대칭성을 고려하여 각 샘플의 Peak 위치 예측 정확도를 계산하는 함수.
    예측 피크 위치가 실제 피크 위치 또는 그 대칭 위치와 일치하면 정답으로 처리합니다.

    Args:
        y_true (np.ndarray): 실제값 배열. (샘플 수, 24, 5, 5) 형태.
        y_pred (np.ndarray): 예측값 배열. (샘플 수, 24, 5, 5) 형태.

    Returns:
        float: Peak 위치 예측 정확도 (%).
    """
    # --- 1. Reshape & Flatten ---
    num_samples = y_pred.shape[0]
    voxel_size = np.prod(y_pred.shape[1:])
    pred_reshaped = y_pred.reshape(num_samples, voxel_size)
    y_true_reshaped = y_true.reshape(num_samples, voxel_size)

    # --- 2. Find 1D indices of max values ---
    flat_indices_pred = np.argmax(pred_reshaped, axis=1)
    flat_indices_true = np.argmax(y_true_reshaped, axis=1)

    # --- 3. Unravel 1D indices back to 3D indices ---
    voxel_shape = y_pred.shape[1:]
    voxel_indices_pred = np.unravel_index(flat_indices_pred, voxel_shape)
    voxel_indices_true = np.unravel_index(flat_indices_true, voxel_shape)

    # --- 4. Stack indices to (N, 3) arrays ---
    peak_indices_pred = np.stack(voxel_indices_pred, axis=1) # (샘플수, 3) [z, y, x]
    peak_indices_true = np.stack(voxel_indices_true, axis=1) # (샘플수, 3) [z, y, x]

    # --- 5. 대칭성을 고려하여 정확도 계산 (수정된 부분) ---

    # 5a. 정답 인덱스의 z, y, x를 분리
    true_z = peak_indices_true[:, 0]
    true_y = peak_indices_true[:, 1]
    true_x = peak_indices_true[:, 2]

    # 5b. 대칭 관계에 있는 정답 인덱스 배열 생성: (z, x, y)
    peak_indices_true_symmetric = np.stack([true_z, true_x, true_y], axis=1)

    # 5c. 두 가지 경우에 대해 일치 여부 확인
    # 경우 1: 예측 인덱스가 원본 정답 인덱스와 일치하는가?
    # (pred_z == true_z) AND (pred_y == true_y) AND (pred_x == true_x)
    matches_direct = np.all(peak_indices_pred == peak_indices_true, axis=1)
    
    # 경우 2: 예측 인덱스가 대칭 정답 인덱스와 일치하는가?
    # (pred_z == true_z) AND (pred_y == true_x) AND (pred_x == true_y)
    matches_symmetric = np.all(peak_indices_pred == peak_indices_true_symmetric, axis=1)

    # 5d. 두 경우 중 하나라도 만족하면 최종 정답으로 처리 (logical OR)
    final_matches = np.logical_or(matches_direct, matches_symmetric)
    
    accuracy = np.mean(final_matches) * 100
    print(f"Peak 위치 (z,y,x) 예측 정확도: {accuracy:.2f}%")
    
    return accuracy

기존버전, 대칭을 고려하지 않았음 (250730 오전)

In [7]:
def calculate_peak_location_accuracy(y_true, y_pred, voxel_type ='assy'):
    """
    각 샘플의 Peak 위치 예측 정확도를 계산하는 함수
    (복셀(Voxel)의 인덱스를 1D로 펼친 후 최대값 인덱스를 찾고 다시 3D 인덱스로 재변환)
    - np.prod(): 리스트의 원소곱을 반환하는 함수이나, 단순히 차원을 곱하기 위해 사용, (24,5,5) -> 24*5*5
    - np.argmax(): 해당 축방향을 따라서 최대값의 인덱스를 반환

    Args:
        y_true (np.ndarray): 실제값 배열. (샘플 수, 24, 5, 5) 형태.
        y_pred (np.ndarray): 예측값 배열. (샘플 수, 24, 5, 5) 형태.
        voxel_type (string): 오차 계산 시 집합체단위 또는 봉단위인지를 명시

    Returns:
        float: Peak 위치 예측 정확도 (%).
    """
    if voxel_type == 'assy':

        # --- 1. Reshape & Flatten ---
            # 예) (24, 5, 5) 복셀을 1차원으로 펼쳐서 (3754, 600) 형태로 변환
        num_samples = y_pred.shape[0] # 3754
        voxel_size = np.prod(y_pred.shape[1:]) # # 24*5*5 = 600
        pred_reshaped = y_pred.reshape(num_samples, voxel_size) # (3754, 600, pred
        y_true_reshaped = y_true.reshape(num_samples, voxel_size) # (3754, 600), true

        # --- 2. Find 1D indices of max values ---
            # 펼쳐진 축(axis=1)을 따라 각 샘플의 최대값 인덱스를 반환
            # 예) [475, 412, 412, ..., 450, 428, 428]: 1차원으로 변환했을때의 최대값 인덱스
        flat_indices_pred = np.argmax(pred_reshaped, axis=1) # (shape: (3754,))
        flat_indices_true = np.argmax(y_true_reshaped, axis=1) # (shape: (3754,))

        # --- 3. Unravel 1D indices back to 3D indices ---
            # 1차원 인덱스를 다시 (z, y, x) 형태의 3차원 복셀 인덱스로 변환
        voxel_shape = y_pred.shape[1:] # y_pred.shape[1:] = (24, 5, 5)
        voxel_indices_pred = np.unravel_index(flat_indices_pred, voxel_shape)
        voxel_indices_true = np.unravel_index(flat_indices_true, voxel_shape)

        # --- 4. Stack indices to (N, 3) arrays ---
            # (샘플수, 3)의 형태이며, 3은 (z,y,x)를 의미
        peak_indices_array_pred = np.stack(voxel_indices_pred, axis=1)
        peak_indices_array_true = np.stack(voxel_indices_true, axis=1)

        # --- 5. Compare indices and calculate accuracy ---
            # 요소별 비교: (3754, 3) 모양의 boolean 배열 생성
            #    [[True, True, True],   <- 0번 샘플: z,y,x 모두 일치
            #     [True, True, False],  <- 1번 샘플: x가 불일치
        comparison_array = (peak_indices_array_pred == peak_indices_array_true)
            # 각 행(axis=1)의 모든 요소가 True인지 확인
            # 결과는 (3754,) shape의 최종 boolean 배열
        matches = np.all(comparison_array, axis=1)
        
        accuracy = np.mean(matches) * 100 # 평균값 * 100: 정확도
        print(f"Peak 위치 예측 정확도: {accuracy:.2f}%")
        
        return accuracy

peak_accuracy = calculate_peak_location_accuracy(y_test, pred_squeezed)

print(f"Peak 위치 예측 정확도: {peak_accuracy:.2f}%")

Peak 위치 예측 정확도: 64.94%
Peak 위치 예측 정확도: 64.94%


In [74]:
peak_indices_array_pred

array([[19,  0,  0],
       [16,  2,  2],
       [16,  2,  2],
       ...,
       [18,  0,  0],
       [17,  0,  3],
       [17,  0,  3]], dtype=int64)

In [76]:
comparison_array = (peak_indices_array_pred == peak_indices_array_true)
# 2. 각 행(axis=1)의 모든 요소가 True인지 확인
#    결과는 (3754,) shape의 최종 boolean 배열
matches = np.all(comparison_array, axis=1)

In [77]:
matches

array([ True,  True,  True, ...,  True,  True,  True])

동일코드이나, 반복문 활용 (검증용)

In [60]:
num_test_data = y_test.shape[0]
peak_indices_list = []

for i in range(num_test_data):
    # i번째 샘플 (24, 5, 5)
    sample_data = pred_squeezed[i]
    
    # 1. i번째 샘플 내에서 최대값의 1차원 인덱스를 찾음
    flat_index = np.argmax(sample_data)
    
    # 2. 1차원 인덱스를 (z, y, x) 3차원 인덱스로 변환
    multi_dim_index = np.unravel_index(flat_index, sample_data.shape)
    
    peak_indices_list.append(multi_dim_index)

# 리스트를 NumPy 배열로 변환
peak_indices_array_loop = np.array(peak_indices_list)

print("\n--- 반복문 방식 결과 ---")
print(f"결과 배열 Shape: {peak_indices_array_loop.shape}")
print("\n첫 5개 샘플의 Peak 복셀 인덱스 (z, y, x):")
print(peak_indices_array_loop[:5])


--- 반복문 방식 결과 ---
결과 배열 Shape: (3754, 3)

첫 5개 샘플의 Peak 복셀 인덱스 (z, y, x):
[[19  0  0]
 [16  2  2]
 [16  2  2]
 [18  0  2]
 [17  2  2]]


### 0. Post Processing: 대각 대칭화 

In [63]:
pred[0].shape
pred_squeezed = np.squeeze(pred)

In [64]:
pred_squeezed.shape

(3754, 24, 5, 5)

In [135]:
pd.DataFrame(pred_squeezed[0,0,:,:])

,0,1,2,3,4
0,0.436258,0.390186,0.252355,0.469171,0.330828
1,0.383254,0.244064,0.456073,0.482506,0.312198
2,0.253718,0.464099,0.516550,0.438659,0.251083
3,0.459658,0.481806,0.450207,0.288065,0.000000
4,0.328062,0.327182,0.248880,0.000000,0.000000


In [70]:
def apply_symmetry(tensor):
  """
  텐서의 마지막 두 축을 기준으로 대각 대칭 행렬로 변환합니다.
  (y, x)와 (x, y) 요소의 평균값을 계산하여 두 위치에 모두 적용합니다.

  Args:
    tensor: (..., H, W) 형태의 NumPy 배열.
            여기서는 (batch_size, 24, 5, 5) 형태의 텐서입니다.

  Returns:
    마지막 두 축이 대각 대칭으로 변환된 새로운 텐서.
  """
  # 마지막 두 축(y, x)을 전치(transpose)합니다.
  # 예를 들어, (0, 1, 2, 3) 축을 (0, 1, 3, 2) 순서로 바꿉니다.
  transposed_tensor = np.transpose(tensor, axes=(0, 1, 3, 2))

  # 원본 텐서와 전치된 텐서를 더한 후 2로 나누어 평균을 구합니다.
  symmetric_tensor = (tensor + transposed_tensor) / 2

  return symmetric_tensor

In [139]:
pred_symmetric = apply_symmetry(pred_squeezed)

In [132]:
import pandas as pd

In [142]:
pd.DataFrame(pred_symmetric[0,0,:,:])

,0,1,2,3,4
0,0.436258,0.386720,0.253036,0.464415,0.329445
1,0.386720,0.244064,0.460086,0.482156,0.319690
2,0.253036,0.460086,0.516550,0.444433,0.249982
3,0.464415,0.482156,0.444433,0.288065,0.000000
4,0.329445,0.319690,0.249982,0.000000,0.000000


SyntaxError: invalid syntax (3697020395.py, line 1)